# Data Modelling

In [1]:
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from helpers.helper_functions import translate_to_file_string
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

inputFile = translate_to_file_string("./data/Data_Preparation_Result.csv")

## Create Spark Session

In [2]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("DataModelling")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- Bundesland: string (nullable = true)
 |-- BundeslandIndex: integer (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- LandkreisIndex: integer (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- AltersgruppeIndex: double (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- GeschlechtIndex: double (nullable = true)
 |-- FallStatus: string (nullable = true)
 |-- FallStatusIndex: double (nullable = true)
 |-- Falldatum: string (nullable = true)

None


## Vorbereitung der Daten

### Filtern der Datensätze
Für das Training dieses Modells ist es sinnvoll nur die Fälle zu betrachten, bei den der Ausgang der Corona-Erkrankung bereits bekannt ist ("GENESEN" oder "GESTORBEN"). Daher werden die Fälle mit noch erkrankten Personen herausgefiltert. Ebenfalls muss der FallStatusIndex neu vergeben werden, damit dieses Feature nur noch die Werte 0 oder 1 enthält.Dies erfolgt im nachfolgenden Abschnitt.

In [3]:
dfNeu = df.filter(df.FallStatus != "NICHTEINGETRETEN").drop("FallStatusIndex").sample(fraction=0.01, withReplacement=False)
dfGestorben = dfNeu.filter(df.FallStatus == "GESTORBEN")
dfGenesen = dfNeu.filter(df.FallStatus == "GENESEN").limit(1000)
dfGesamt = dfGestorben.union(dfGenesen)


In [4]:
dfGesamt.groupBy("FallStatus").count().show()

+----------+-----+
|FallStatus|count|
+----------+-----+
|   GENESEN| 1000|
| GESTORBEN|  869|
+----------+-----+



### FallStatusIndex

In [5]:
# Wollten eigentlich den Indexer mitgeben in die Pipeline. Dies führt aber zum Fehler, dass er das Label nicht kennt.
indexer = StringIndexer(inputCol="FallStatus", outputCol="FallStatusIndex")

### Aufbau des Feature-Vectors

In [6]:
assembler =  VectorAssembler(outputCol="features", inputCols=["GeschlechtIndex","AltersgruppeIndex", "LandkreisIndex","BundeslandIndex"]) #"LandkreisIndex"

### Splitten in Trainings und Testdaten

In [7]:
splits = dfGesamt.randomSplit([0.8, 0.2 ], 345678)
trainingVector = splits[0]
testVector = splits[1]

trainingVector.limit(10).show()

+-----------------+---------------+------------------+--------------+------------+-----------------+----------+---------------+----------+----------+
|       Bundesland|BundeslandIndex|         Landkreis|LandkreisIndex|Altersgruppe|AltersgruppeIndex|Geschlecht|GeschlechtIndex|FallStatus| Falldatum|
+-----------------+---------------+------------------+--------------+------------+-----------------+----------+---------------+----------+----------+
|Baden-Württemberg|              8|       LK Biberach|          8426|        A80+|              3.0|         M|            1.0| GESTORBEN|2021-04-08|
|Baden-Württemberg|              8|  LK Bodenseekreis|          8435|        A80+|              3.0|         W|            0.0| GESTORBEN|2021-01-10|
|Baden-Württemberg|              8|  LK Bodenseekreis|          8435|        A80+|              3.0|         W|            0.0| GESTORBEN|2021-01-11|
|Baden-Württemberg|              8|      LK Göppingen|          8117|     A60-A79|              2.0|

## Modellierung
### Support Vector Machine

In [8]:
lr = LinearRegression(maxIter=10, featuresCol="features", labelCol="FallStatusIndex")

### Pipeline

In [9]:
pipeline = Pipeline(stages=[indexer,assembler, lr])

### Parametertuning
Eine wichtige Aufgabe beim Machine Learning ist die Auswahl des geeigneten Modells bzw. die passenden Paramter für ein Modell herauszufinden. Letzteres wird auch Parametertuning genannt. Die in Pyspark enthaltene MLLib bietet speziell hierfür ein entsprechende Tooling. Und zwar kann ein CrossValidator bzw. ein TrainValidationSplit verwendet werden. Voraussetzung sind ein Estimator (ein Modell oder eine Pipeline), ein Paramter-Grid und eine Evaluator. Dies ist auch im Zusammenhang mit dem Thema Cross-Validation zu sehen. (Apache Spark 2020a)

In [10]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.3,0.01]) \
    .addGrid(lr.elasticNetParam, [0.8])\
    .build()

### Cross-Validation
Wie eben erwähnt, benötigt der Cross-Validator einen Evaluator. Letzterer ist zu wählen, abhängig von dem jeweilligen Modell und Anwendungsfall. (Apache Spark 2020a) In diesem Fall wird ein BinaryClassificationEvaluator angewendet. Dieser eignet sich besonders für binäre Werte. (Apache Spark 2021a) Da Geschlecht, in diesem Fall, der FallStatus 0 oder 1 annehmen kann, bietet er sich hier besonders an.

In [11]:
# Definition des Evaluators
evaluator = BinaryClassificationEvaluator(labelCol="FallStatusIndex",rawPredictionCol="prediction", metricName="areaUnderPR")

In [12]:
# Definition des Cross-Validators 
# num-Folds gibt an in wie viele Datensatz-Paare die Datensätze aufgeteilt werden.
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2,
                          parallelism=2)

#### Durchführung

In [13]:
# Anpassung des Modells und Auswahl der besten Parameter
cvModel = crossval.fit(trainingVector)

### Testen des Modells

In [14]:
predictions = cvModel.transform(testVector)
predictions.show()

+--------------------+---------------+--------------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+-------------------+
|          Bundesland|BundeslandIndex|           Landkreis|LandkreisIndex|Altersgruppe|AltersgruppeIndex|Geschlecht|GeschlechtIndex|FallStatus| Falldatum|FallStatusIndex|            features|         prediction|
+--------------------+---------------+--------------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+-------------------+
|   Baden-Württemberg|              8|  LK Schwäbisch Hall|          8127|        A80+|              3.0|         M|            1.0| GESTORBEN|2021-04-16|            1.0|[1.0,3.0,8127.0,8.0]| 0.7301230115004156|
|              Bayern|              9|LK Dingolfing-Landau|          9279|        A80+|              3.0|         W|            0.0| GESTORBEN|2020-03-2

In [15]:
predictions.groupBy("FallStatusIndex", "prediction").count().show()


+---------------+-------------------+-----+
|FallStatusIndex|         prediction|count|
+---------------+-------------------+-----+
|            1.0| 0.5281253598544933|    1|
|            0.0| 0.5330925342108636|    5|
|            1.0|0.48906698206988375|    1|
|            0.0| 0.6885810465376208|    2|
|            0.0| 0.6860974593594358|    2|
|            1.0| 0.5082566624290121|    2|
|            0.0|0.11667929502809347|    7|
|            0.0| 0.0726537428871137|    2|
|            0.0| 0.8880951110053579|    1|
|            0.0| 0.2870693304239616|    4|
|            1.0| 0.7450245345695263|    1|
|            1.0| 0.5306089470326785|    3|
|            1.0|0.46671469746621747|    1|
|            0.0| 0.6960318080721764|    2|
|            0.0|0.28458574324577646|    1|
|            0.0|   0.67367952346851|    1|
|            0.0| 0.9147355528990415|    1|
|            1.0|0.11419570784990833|    1|
|            0.0| 0.2647170458202953|    4|
|            1.0| 0.712737901253

## Modell - Evaluation

### Area Under Roc

In [16]:
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))

Test Error 0.27901247493113035


### BinaryClassificationMetrics

Bei dem untersuchten Label (FallStatus mit den Ausprägungen Verstorben und Genesen) handelt es sich um ein einen BinaryClasificator. Er kann die Werte 0 und 1 annehmen. Für die Modellevaluation sind daher die BinaryClassificationMetrics zu verwenden. (Apache Spark 2021i)

In [17]:
predictionAndLabels = predictions.select("prediction", "FallStatusIndex").rdd.map(lambda p: [p[0], float(p[1])]) # Map to RDD prediction|label

In [18]:
# Instanzieire das BinaryClassificationMetrics-Objekt
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Fläche unter der Precision-recall Curve

print("Area under PR = %s" % metrics.areaUnderPR)

# Fläche unter der ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 0.7209875250688697
Area under ROC = 0.8516217870257039


### Multiclass classification Metrics
In den meißten Fällen können auch Multiclass Classification Metrics bei Binary Classifaction Problemen angewandt werden.

In [19]:
# Instantiate metrics object
metrics = MulticlassMetrics(predictionAndLabels)


In [20]:
# Overall statistics
precision = metrics.precision(1.0)
recall = metrics.recall(1.0)
f1Score = metrics.fMeasure(1.0)


print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)



labels = predictions.select("FallStatusIndex").rdd.map(lambda lp: lp.FallStatusIndex).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

# Weighted stats
print("Weighted recall = %s" % metrics.weightedRecall)
print("Weighted precision = %s" % metrics.weightedPrecision)
print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)

TypeError: 'float' object is not callable